In [1]:
import requests
import json
import pandas as pd
import datetime
import time
import pandas_datareader.data as web
from requests.adapters import HTTPAdapter, Retry

pd.core.common.is_list_like = pd.api.types.is_list_like

In [2]:
retry_strategy = Retry(
  total=3,
  backoff_factor=0.1,
  status_forcelist=[ 500, 502, 503, 504 ]
)

adapter = HTTPAdapter(max_retries=retry_strategy)
http = requests.Session()
http.mount("https://", adapter)
http.mount("http://", adapter)

В качестве данных буду брать дневные значения цены криптовалют (open, high, low, close).

Вытягиваем id и названия всех монет которые есть на coinmarketcap

In [3]:
api_url_id = 'https://api.coinmarketcap.com/data-api/v1/cryptocurrency/map'
response = http.get(api_url_id)

def get_coinmarketcap_info(url: str):
    for item in response.json()['data']:
        identificator = item['id']
        name = item['name']
        symbol = item['symbol']
        is_active = item['is_active']
        yield identificator, name, symbol, is_active
cols = ["id", "name", "symbol", "is_active"]
dataframe = (elem for elem in get_coinmarketcap_info(api_url_id))
df_names = pd.DataFrame(dataframe, columns= cols)

In [4]:
print(df_names.shape)
df_names.to_csv('data/list_of_names.csv')
df_names.head(5)

(8938, 4)


,id,name,symbol,is_active
0,1,Bitcoin,BTC,1
1,2,Litecoin,LTC,1
2,3,Namecoin,NMC,1
3,4,Terracoin,TRC,1
4,5,Peercoin,PPC,1


Идея о том чтобы пробегать по всем id для выявления топа по объему за последние сутки отпала

Просто вытянем id интересных монеток

In [5]:
crypto_list = ['BTC', 'ETH', 'BNB', 'XRP', 'DOGE', 'ADA', 'MATIC', 'DAI', 'DOT', 'LTC']
print(len(crypto_list))

10


In [6]:
final_list = df_names.loc[df_names['symbol'].isin(crypto_list)].reset_index(drop=True)
final_list

,id,name,symbol,is_active
0,1,Bitcoin,BTC,1
1,2,Litecoin,LTC,1
2,52,XRP,XRP,1
3,74,Dogecoin,DOGE,1
4,1027,Ethereum,ETH,1
5,1839,BNB,BNB,1
6,2010,Cardano,ADA,1
7,3890,Polygon,MATIC,1
8,4943,Dai,DAI,1
9,6636,Polkadot,DOT,1


In [7]:
DAYS_FROM_NOW_FOR_PARSE = 2200

# Дата окончания - текущий день
tmp_date_now = datetime.date.today()

# Выявлено методом тыка, что api выдает ровно 3 часа ночи
date_now = datetime.datetime(tmp_date_now.year, tmp_date_now.month, tmp_date_now.day, 3, 0)

# Дата старта
date_start = date_now - datetime.timedelta(days=DAYS_FROM_NOW_FOR_PARSE)

# Тестируем что переводит в unix время
print("date_now", date_now)
print("unix_timestamp date_now:", int(time.mktime(date_now.timetuple())))
print("date_start", date_start)

unix_date_start = int(time.mktime(date_start.timetuple()))
unix_date_now = int(time.mktime(date_now.timetuple()))

date_now 2022-12-28 03:00:00
unix_timestamp date_now: 1672185600
date_start 2016-12-19 03:00:00


Возникла проблема: если вытягивать данные с общедоступного API (которым я и пользуюсь), то есть ограничение на 100 строк.
То есть, если мы хотим тянуть самые свежие данные, то одним запросом можем вытянуть только 100 последних дней.
Иначе интервал разбивается на одинаковые отрезки.

Пробежим циклом для каждой монетки и склеим

In [8]:
api_url_cryptocurrency = 'https://api.coinmarketcap.com/data-api/v3/cryptocurrency/historical?id=1&convertId=2781&timeStart=1420070400&timeEnd=1637712000'

def unix_time(tmp: datetime.datetime) -> int:
    return int(time.mktime(tmp.timetuple()))

def get_coinmarketcap_cryptocurrency(id: int, start: int, end: int):
    url = f'https://api.coinmarketcap.com/data-api/v3/cryptocurrency/historical?id={id}&convertId=2781&timeStart={start}&timeEnd={end}'
    response = http.get(url)
    data = []
    for item in response.json()['data']['quotes']:
        date = item['timeOpen']
        open_ = item['quote']['open']
        close = item['quote']['close']
        volume =item['quote']['volume']
        marketCap =item['quote']['marketCap']
        date2 = item['quote']['timestamp']
        high = item['quote']['high']
        low = item['quote']['low']
        data.append([date, open_, close, volume, marketCap, date2, high, low])
    return data

def get_data(names_df, start, end, max_delta=99):
    cols = ["timeOpen", "open", "close", "volume", "marketCap", "timestamp", "high", "low"]
    # Очень аккуратно бегаем по датам
    # Делаем запросы по времени от start до start + 99 дней или до start + n, где n - дней до текущей даты
    for i in range(len(names_df)):
        df_temp = pd.DataFrame()
        start_tmp = start
        while True:
            tmp_delta = max_delta
            unix_tmp_start = unix_time(start_tmp)
            if (end - start_tmp).days <= 0: break 
            elif (end - start_tmp).days > max_delta:
                tmp = max_delta
                unix_tmp_end = unix_time(start_tmp+datetime.timedelta(days=(tmp-1)))
                df_temp = pd.concat([df_temp, pd.DataFrame(get_coinmarketcap_cryptocurrency(names_df.iloc[i][0], unix_tmp_start, unix_tmp_end), columns= cols)])
            else:
                tmp = (end - start_tmp).days
                unix_tmp_end = unix_time(start_tmp+datetime.timedelta(days=(tmp)))
                df_temp = pd.concat([df_temp, pd.DataFrame(get_coinmarketcap_cryptocurrency(names_df.iloc[i][0], unix_tmp_start, unix_tmp_end), columns= cols)])
            start_tmp+=datetime.timedelta(days=tmp)
        df_temp.reset_index(drop=True).to_csv("data/" + names_df.iloc[i][2]+'.csv')
        print(names_df.iloc[i][1] + ' saved!')

In [9]:
get_data(final_list, date_start, date_now)

Bitcoin saved!
Litecoin saved!
XRP saved!
Dogecoin saved!
Ethereum saved!
BNB saved!
Cardano saved!
Polygon saved!
Dai saved!
Polkadot saved!


Протестировал метод для парсинга 6 последних лет - все ок


Один из вариантов развития: 
    
1. Пытаемся открыть файл: если его не существует то парсим с начальной даты
2. Если открылся, то по последней строке смотрим последнюю дату. Если она меньше текущей, то дозаписываем файл.
3. Обновляем графики

По технической части: переезжаем на httpx